In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing 

# Load data

In [ ]:
housing= pd.read_csv("Data/housing.csv")
anscombe_df = pd.read_csv("Data/anscombe.csv")
print(housing.head())
print("------")
print(anscombe_df.head())

In [ ]:
housing_df = housing.copy()

In [ ]:
housing_df.info()

In [ ]:
housing_df.describe()

# Analysis

In [ ]:
# seeing the boxplot of housing_median_age vs median house value
plt.figure(figsize=(15,8))
sns.boxplot(housing_df["housing_median_age"], housing_df["median_house_value"])

In [ ]:
plt.figure(figsize=(15,8))
sns.distplot(housing_df["median_house_value"])

In [ ]:
housing_df.hist(bins=50, figsize=(15,15))

* The attributes have different scales. It is recommended to rescale all the attributes.
* <b><u>The medium house value</u></b> has a sudden peak around 500000, which is very different from others.
* It is recommended to remove these data in training the model.
* <b><u>The medium income</u></b> is centered around 3, where the unit is unknown. Probably, 3 means $300,000.

In [ ]:
housing_df["ocean_proximity"].value_counts()

In [ ]:
housing_df["ocean_proximity"].value_counts().plot(kind="bar")


In [ ]:
# Box plot for median house value vs ocean proximity
sns.boxplot(x="ocean_proximity", y="median_house_value", data=housing_df)

In [ ]:
sns.pairplot(housing_df, x_vars=['housing_median_age', 'population', 'households', 'median_income'],y_vars ='median_house_value',hue = 'ocean_proximity')

In [ ]:
map_dict = {'<1H OCEAN':0, 'INLAND':1, 'ISLAND':2, 'NEAR BAY':3, 'NEAR OCEAN':4}
def map_fea(x):
    return map_dict[x]
    
housing_df["ocean_proximity"] = housing_df["ocean_proximity"].apply(map_fea)
housing_df.head()

#### Add a new categorical feature in order to split the dataset properly
* income_data

In [ ]:
housing_df['income_data'] = np.ceil(housing_df['median_income']/1.5)
housing_df['income_data'].where(housing_df['income_data']<5, 5, inplace=True)

In [ ]:
plt.hist(housing_df.income_data)


* Most people has annual income more than $300,000

In [ ]:
housing_df.income_data.value_counts()/len(housing_df.income_data)

Prepare Data for Machine Learning Algorithm
* split the data into train and test sets based on the new feature

* Method 1: using the package train_test_split without the new feature
* Method 2: using the package StratifiedShuffleSplit with the new feature

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data4train,data4test = train_test_split(housing_df, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state = 42)

In [ ]:
for train_index, test_index in split.split(housing_df, housing_df['income_data']):
    train_set = housing_df.loc[train_index]
    test_set  = housing_df.loc[test_index] 

In [ ]:
def table4income_cat(dataset,df,label):
    df[label]=pd.Series(dataset['income_data'].value_counts()/len(dataset['income_data']))
    return df

In [ ]:
df = pd.DataFrame()
df = table4income_cat(train_set,df,'All_set')
df = table4income_cat(train_set,df,'train_set_Shuff')
df = table4income_cat(test_set,df,'test_set_Shuff')
df = table4income_cat(data4train,df,'train_set_split')
df = table4income_cat(data4test,df,'test_set_split')
df

* It is shown above that the <u><b>StratifiedShuffleSplit</b></u> method works a little bit better as the income category propotions in train data and test data are closer to that in the all dataset.

* After split the data into train and test, we delete the newly added feature

In [ ]:
# delete the new feature
for set_ in (housing_df, data4train, data4test, train_set, test_set):
    set_.drop("income_data",axis=1,inplace=True)

In [ ]:
data4train.columns

In [ ]:
housing4train = train_set.copy()

In [ ]:
housing4train.head()

In [ ]:
# California maping

In [ ]:
housing4train.plot(kind='scatter', x='longitude', y='latitude', alpha=0.3,
         s=housing4train['population']/100, label='population',   # set symbol size on population
         c=housing4train['median_house_value'],                  #  set symbol color on house value    
         cmap=plt.get_cmap('jet'),      
         colorbar=True,
         figsize=(10,7))
plt.legend()

* Population and location(how close to the ocean) affects the housing value

## Correlation

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(housing_df.corr(),annot=True,fmt=".3f")

In [ ]:
corr_matrix = housing4train.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# In dataset form instead of map
corr_matrix

In [ ]:
df = corr_matrix.tail(2).T
df

In [ ]:
df.sort_values(by='median_house_value',inplace=True) 
df

#### * Able to see that the large correlations within median house value and income, total_rooms and house age

## Data for ML Algorithm

In [ ]:
x_train = data4train.drop("median_house_value",axis=1)
y_train = data4train["median_house_value"].copy()

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_test = data4test.drop("median_house_value",axis=1)
y_test = data4test["median_house_value"].copy()

In [ ]:
x_test.shape, y_test.shape

## Look for missing data:


In [ ]:
#missing data
def report_missing_data(dataset):
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = dataset.isnull().sum()/total 
        
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    missing_data.plot(kind='bar',y='Total',figsize=(6,4),fontsize=9)
    print(missing_data)

In [ ]:
report_missing_data(x_test)

In [ ]:
report_missing_data(x_train)

In [ ]:
x_train = train_set.drop("median_house_value",axis=1)
y_train = train_set["median_house_value"].copy()

x_test = test_set.drop("median_house_value",axis=1)
y_test = test_set["median_house_value"].copy()

In [ ]:
report_missing_data(x_train)

## Clean Data

In [ ]:
x_train['total_bedrooms'].isnull().sum()

In [ ]:
missing_feature = pd.DataFrame(x_train.isnull().sum().sort_values(ascending=False)).index[0]
missing_feature

* Only the total bedrooms contains missing values.

In [ ]:
median=x_train[missing_feature].median()
x_train[missing_feature]=x_train[missing_feature].replace(np.nan,median)
x_train[missing_feature].isnull().sum()

In [ ]:
median=x_test[missing_feature].median()
x_test[missing_feature]=x_test[missing_feature].replace(np.nan,median)
x_test[missing_feature].isnull().sum()

One-hot encoding

In [ ]:
housing_num = x_train.drop("ocean_proximity",axis=1)
num_attribs = list(housing_num)

In [ ]:
num_attribs

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')  
from sklearn.pipeline import FeatureUnion
#CategoricalEncoder(encoding='onehot-dense')


from sklearn.base import BaseEstimator,TransformerMixin
#select columns and transit to array

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out
        
class DataFrameSelector(BaseEstimator,TransformerMixin):
    def __init__(self,feature_names):
        self.feature_names = feature_names
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X[self.feature_names].values
    

# build pipelines
cat_attribs = ['ocean_proximity']
num_attribs = list(housing_num)

num_pipeline = Pipeline([
               ('selector',DataFrameSelector(num_attribs)),      
               ('std_scaler',StandardScaler()), 
                ]) 

# build categorical pipeline
cat_pipeline = Pipeline([
                  ('selector',DataFrameSelector(cat_attribs)),
                  ('cat_encoder',CategoricalEncoder(encoding='onehot-dense')),
              ])


# concatenate all the transforms using "FeatureUnion"
pipelines = FeatureUnion(transformer_list=
                             [ 
                              ('num_pipeline',num_pipeline),
                              ('cat_pipeline',cat_pipeline),
                             ])

In [ ]:
x_train_prepared = pipelines.fit_transform(x_train)

In [ ]:
x_train_prepared.shape

## Train the model

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

def model_performace(model,X,y):
    model.fit(X,y)
    pred = model.predict(X)
    scores = np.sqrt(mean_squared_error(pred,y))
    
    print("scores:",scores)
    print("Mean:",scores.mean())
    print("Standard Deviation:",scores.std())
    return model,pred

In [ ]:
def plot_pred_true(ypred,ytrue):
    df = pd.DataFrame([ypred,ytrue]).T
    df.columns=['pred','true']
    plt.scatter(df['pred'],df['true'])

# Compare models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
lr.score(x_test, y_test)

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(x_train, y_train)
dt.score(x_test, y_test)

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

#### ------------------------------------------------------------------------------------------
#### * It can be seen from the figure that the effect of random forest model is the best, far better than the other two models. The effect of using decision tree is the worst.
#### ------------------------------------------------------------------------------------------

## Linear Regression

In [ ]:
lr = LinearRegression()
lr_model,ypred = model_performace(lr,x_train_prepared, y_train) 

## Decision Tree

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(x_train, y_train)
dt.score(x_test, y_test)

In [ ]:
tree = DecisionTreeRegressor() 
tree_model,ypred= model_performace(tree,x_train_prepared, y_train)

## Random Forest

In [ ]:
forest = RandomForestRegressor()
forest_model,y_pred = model_performace(forest,x_train_prepared,y_train)

### Testing

In [ ]:
housing_df1 = housing.copy() 

In [ ]:
housing_df1.info()


In [ ]:
housing_df1.plot.scatter(x = "housing_median_age", y = "population")

In [ ]:
housing_df1 = housing_df1.loc[housing_df["population"]<20000]

In [ ]:
housing_df1.info()

In [ ]:
housing_df1.plot.scatter(x = "housing_median_age", y = "population")

In [ ]:
housing_df1.columns

In [ ]:
housing_df1["house_pop"] = housing_df1["households"]/housing_df1["population"]

In [ ]:
housing_df1 = pd.get_dummies(housing_df1, columns=["ocean_proximity"])

In [ ]:
housing_df1.columns
housing_df1_norm = housing_df1.copy()

In [ ]:
scaler = MinMaxScaler()
column_names_to_normalize = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'house_pop']
x = housing_df1_norm[column_names_to_normalize].values
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = housing_df1_norm.index)
housing_df1_norm[column_names_to_normalize] = df_temp

In [ ]:
housing_df1_norm.head()

In [ ]:
# test model 1

In [ ]:
m1_data = housing_df1.copy()
m1Y = m1_data['median_house_value']
m2_data = housing_df1.copy()
m2_data.drop(columns=['total_rooms',
       'total_bedrooms','population','households'],inplace=True)
m1_data.drop(columns=['median_house_value','total_rooms',
       'total_bedrooms','population','households'],inplace=True)
m1_data.columns

In [ ]:
m1X_train, m1X_test, m1y_train, m1y_test = train_test_split(m1_data, m1Y, test_size=0.2,random_state=1)

In [ ]:
model1 = linear_model.LinearRegression()
model1.fit(m1X_train, m1y_train)

In [ ]:
m1y_pred = model1.predict(m1X_test)

In [ ]:
print(metrics.mean_squared_error(m1y_test, m1y_pred))
print(np.sqrt(metrics.mean_squared_error(m1y_test, m1y_pred)))

In [ ]:
pd.DataFrame(zip(m1_data.columns,model1.coef_))

In [ ]:
pd.DataFrame(zip(m1y_test,m1y_pred),columns=['Actual','Predicted'])

* As you can see, the sample predictions are not very good: such as the first one: it is off by (378000-326749)/378000~=13%.

In [ ]:
model1.score(m1X_train, m1y_train)

In [ ]:
m2_data.rename(columns={'ocean_proximity_<1H OCEAN': 'ocean_proximity1', 'ocean_proximity_NEAR BAY': 'ocean_proximity2', 'ocean_proximity_NEAR OCEAN': 'ocean_proximity3'}, inplace=True)

In [ ]:
lm2 = smf.ols(formula='median_house_value ~ longitude + latitude + housing_median_age  + median_income + house_pop + ocean_proximity1 + ocean_proximity_INLAND + ocean_proximity2 + ocean_proximity3', data=m2_data).fit()

In [ ]:
lm2.summary()

## Training models:

#### Decision Tree

In [ ]:
model_dt = make_pipeline(preprocessing.StandardScaler(),DecisionTreeRegressor(random_state=0))
scores = cross_val_score(model_dt, m1X_train, m1y_train, cv=10)

model_dt.fit(m1X_train, m1y_train)
pred = model_dt.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic={}

dic['DecisionTree'] = (RMSE,scores.mean() *100)

print("Decision Tree")
print("---------------------")
print("MSE:", MSE, "RMSE:", RMSE)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

#### Linear Regression

In [ ]:
model_lr = make_pipeline(preprocessing.StandardScaler(),linear_model.LinearRegression())
scores = cross_val_score(model_lr, m1X_train, m1y_train, cv=10)

model_lr.fit(m1X_train, m1y_train)
pred = model_lr.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['LinearRegression'] = (RMSE,scores.mean() *100)

print("Linear Regression")
print("---------------------")
print("MSE:", MSE, "RMSE:", RMSE)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

#### Random Forest

In [ ]:
model_rf = make_pipeline(preprocessing.StandardScaler(),RandomForestRegressor(max_depth=50, random_state=0))
scores = cross_val_score(model_rf, m1X_train, m1y_train, cv=10)

model_rf.fit(m1X_train, m1y_train)
pred = model_rf.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['Random Forest'] = (RMSE,scores.mean() *100)

print("Random Forest")
print("---------------------")
print("Depth:50 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

In [ ]:
test = model_rf.fit(m1X_train, m1y_train)

In [ ]:
pred = test.predict(m1X_test)

In [ ]:
print("RMSE: ", np.sqrt(metrics.mean_squared_error(m1y_test, pred)))

## Output all results

In [ ]:
accuracy_model = pd.DataFrame(dic,index=['RMSE','Training_Accuracy'])
accuracy_model = accuracy_model.T
accuracy_model

In [ ]:
accuracy_model.sort_values(by=['Training_Accuracy'],inplace=True)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(8,5))
sns.barplot(x=accuracy_model.index,y='Training_Accuracy',data=accuracy_model,ax=ax1,palette='Blues')
plt.title("Models_comparision")
plt.savefig("static/images/model_comparision.png")